<img src="dataset_5-cover.jpg" alt="Circular Image" style="border-radius: 50%; display: block; margin: 0 auto; width: 200px; height: 200px;">
<h1>Superstore Marketing Campaign Dataset</h1>
<p><i>Sample customer data for analysis of a targeted Membership Offer</i></p>
<h2>About Dataset</h2>
<p><b>Context</b>: A superstore is planning for the year-end sale. They want to launch a new offer - gold membership, that gives a 20% discount on all purchases, for only $499 which is $999 on other days. It will be valid only for existing customers and the campaign through phone calls is currently being planned for them. The management feels that the best way to reduce the cost of the campaign is to make a predictive model which will classify customers who might purchase the offer.</p>
<h2>Data Source:</h2>
<p><b>Kaggle: </b> <a href="https://www.kaggle.com/datasets/ahsan81/superstore-marketing-campaign-dataset">Click Here!!!</a></p>

In [ ]:
import duckdb
con = duckdb.connect("data/db/superstore.duckdb", read_only=True)

In [2]:
query = """
SELECT COUNT(*)
FROM marketing_dataset
"""
con.sql(query).df()

,count_star()
0,2240
